In [1]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import os

In [2]:
dlib.DLIB_USE_CUDA

True

In [3]:
# 移除.ipynb_checkpoints資料夾, 避免讀取圖片時發生錯誤
import shutil

nowpath = os.getcwd()
try:   
    shutil.rmtree(nowpath +'/traing_set/akane/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/traing_set/neru/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/testing_set/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/traing_set/rika/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/traing_set/risa/.ipynb_checkpoints')
except:
    pass
try:   
    shutil.rmtree(nowpath +'/traing_set/yui/.ipynb_checkpoints')
except:
    pass

In [4]:
data_path = "./training_set"

x_data_list = []
y_data_list = []
for roots, dirs, files in os.walk(data_path):
    if dirs == []:
        for each in files:
            if each.find('check') == -1:
                x_data_list.append(os.path.join(roots, each))
                y_data_list.append(roots.split("/")[-1])

In [5]:
data_path = "./testing_set"
names=[]
test_list = []
for roots, dirs, files in os.walk(data_path):
        for each in files:
            if each.find('check') == -1:
                names.append(each.split(".")[0])
                test_list.append(os.path.join(roots, each))

In [6]:
face_cascade = cv2.CascadeClassifier('/home/kaneyxx/miniconda3/envs/tf2/lib/python3.7/site-packages/cv2/data/haarcascade_frontalface_default.xml')

In [7]:
def detect_train_cv2(filename):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,
                                          scaleFactor=1.1,
                                          minNeighbors=3,)
    count = 0
    for (x,y,w,h) in faces:
#         img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
        f = cv2.resize(img[y:y+h+30, x:x+w+30], (200, 200))
        sets1 = filename.split("/")
        sets2 = filename.split("/")[3].split(".")
        name = "./face_cv2_train/" + sets1[2] + "/" + sets2[0] + "_" + str(count) + ".png"
        cv2.imwrite(name, f)
        count += 1

In [8]:
for path in x_data_list:
    detect_train_cv2(path)

In [9]:
def detect_test_cv2(filename):
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,
                                          scaleFactor=1.1,
                                          minNeighbors=3,)
    if len(faces) > 0:
        count = 0
        for (x,y,w,h) in faces:
            f = cv2.resize(img[y:y+h+30, x:x+w+30], (200, 200))
            sets = filename.split("/")[2].split(".")
            name = "./face_cv2_test/" + sets[0] + "_" + str(count) + ".png"
            cv2.imwrite(name, f)
            count += 1

In [10]:
for path in test_list:
    detect_test_cv2(path)

In [11]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

In [12]:
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb

In [13]:
fa = FaceAligner(predictor, desiredFaceWidth=200)

In [14]:
def detect_face_train(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)
    sets1 = path.split("/")
    sets2 = path.split("/")[3].split(".")
    if len(faces) > 0:
        count = 1
        for face in faces:
            try:
                (x, y, w, h) = rect_to_bb(face)
                faceOrig = imutils.resize(img[y : y+h, x : x+w], width=200)
                faceAligned = fa.align(img, gray, face)
                orig_name = "./face_dlib_train/" + sets1[2] + "/" + sets2[0] + "_orig_" + str(count) + "." + sets2[1]
                cv2.imwrite(orig_name, faceOrig)
                aligned_name = "./face_dlib_train/" + sets1[2] + "/" + sets2[0] + "_align_" + str(count) + "." + sets2[1]
                cv2.imwrite(aligned_name, faceAligned)
                count += 1
            except:
                pass

In [15]:
for i in x_data_list:
    detect_face_train(i)

In [16]:
def detect_face_test(path):
    count = 1
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)
    sets = path.split("/")[2].split(".")
    for face in faces:
        try:
            (x, y, w, h) = rect_to_bb(face)
            faceOrig = imutils.resize(img[y : y+h, x : x+w], width=200)
            faceAligned = fa.align(img, gray, face)
            orig_name = "./face_dlib_test/origin/" + sets[0] + "_orig_" + str(count) + "." +sets[1]
            cv2.imwrite(orig_name, faceOrig)
            aligned_name = "./face_dlib_test/aligned/" + sets[0] + "_align_" + str(count) + "." +sets[1]
            cv2.imwrite(aligned_name, faceAligned)
            count += 1
        except:
            pass

In [17]:
for i in test_list:
    detect_face_test(i)

In [18]:
cnn_face_detector = dlib.cnn_face_detection_model_v1("./mmod_human_face_detector.dat")

In [19]:
def detect_cnn_test(path):
    count = 1
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_cnn = cnn_face_detector(gray, 1)
    sets = path.split("/")[2].split(".")
    for face in faces_cnn:
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
        try:
            faceOrig = imutils.resize(img[y : y+h, x : x+w], width=200)
            orig_name = "./face_cnn_test/" + sets[0] + "_" + str(count) +  "_" + "." +sets[1]
            cv2.imwrite(orig_name, faceOrig)
            count += 1
        except:
            pass

In [21]:
for i in test_list:
    detect_cnn_test(i)

In [22]:
data_path = "./face_cnn_test/"
files = os.listdir(data_path)

n = 0
for i in files:
    oldname = data_path + files[n]
    newname = data_path + files[n].split("_")[0] + ".png"
    os.rename(oldname, newname)
    n += 1

In [29]:
data_path1 = "./face_cnn_test/"
files1 = os.listdir(data_path1)

data_path2 = "./testing_set/"
files2 = os.listdir(data_path2)

diff = []
for i in files2:
    if i not in files1:
        diff.append(i)

In [33]:
def detect_cnn_train(path):
    count = 1
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_cnn = cnn_face_detector(gray, 1)
    sets1 = path.split("/")
    sets2 = path.split("/")[3].split(".")
    for face in faces_cnn:
        x = face.rect.left()
        y = face.rect.top()
        w = face.rect.right() - x
        h = face.rect.bottom() - y
        try:
            faceOrig = imutils.resize(img[y : y+h, x : x+w], width=200)
            orig_name = "./face_cnn_train/" + sets1[2] + "/" + sets2[0] + "_" + str(count) +  "_" + "." +sets2[1]
            cv2.imwrite(orig_name, faceOrig)
            count += 1
        except:
            pass

In [34]:
for path in x_data_list:
    detect_cnn_train(path)